In [1]:
import sklearn as sk
from sklearn.model_selection import train_test_split
from sklearn.model_selection import GridSearchCV
import numpy as np
import pandas as pd
import xgboost as xgb
from sklearn.metrics import roc_auc_score
import time
from sklearn.model_selection import KFold
import warnings
from sklearn.model_selection import cross_val_score

In [2]:
start_time = time.time()
df = pd.read_csv('C:\\users\\user\\cc_sample.txt', sep=';',index_col=False, decimal=',') 
print(-start_time + time.time()," sec")

2.807321310043335  sec


In [3]:
#Get Dataframe as input, return DataFrame with filled nulls
def fill_null(x,s):
    for i in x.columns:
        x[i] = x[i].fillna(s)
    return x

#Check type of variable and return numeric or cat
def check_type(x):
    from pandas.api.types import is_string_dtype
    from pandas.api.types import is_numeric_dtype   
    #Удаляем пустые значения
    x = x[x.notnull()]
    #Если число различных значений меньше 4, то тип-категориальный
    if x.nunique()<=4: return 'cat'
    elif is_numeric_dtype(x): return 'numeric'
    else: return 'cat'
    
#Input: DataFrame
#Output: DataFrame with one-hot variables
def cat_to_one_hot(x):
    for col in x.columns:
        if check_type(x[col])=='cat':
            tmp = pd.get_dummies(x[col],prefix=x[col].name,drop_first=True)
            for i in tmp.columns:
                x[i] = tmp[i]
            del x[col]
    return x

def gini(model,x,y):
    gini =  2*roc_auc_score(y,model.predict_proba(x)[:,1])-1
   # print('GINI = ',2*roc_auc_score(y,model.predict_proba(x)[:,1])-1)   
    return gini

def build_xgb_model(params_grid,x,y):
    xgb_model = xgb.XGBClassifier(base_score=0.5, colsample_bylevel=1, colsample_bytree=1,
                               gamma=0, learning_rate=0.1, max_delta_step=0, max_depth=4,
                               min_child_weight=1, missing=None, n_estimators=500, nthread=-1,
                               objective='binary:logistic', reg_alpha=30, reg_lambda=1,
                               scale_pos_weight=1, seed=42, silent=True, subsample=1,tree_method='gpu_hist')
    kfold = KFold(n_splits=3, shuffle=True)
    clf = GridSearchCV(xgb_model,params_grid, 
                            verbose=2, 
                            scoring='roc_auc',
                            cv=kfold,
                            n_jobs=-1)
    clf.fit(x,y)
    print(-start_time + time.time()," sec")
    warnings.simplefilter('ignore', FutureWarning)
    res = pd.DataFrame(clf.cv_results_)
    res["dev_gini"] = 2*res["mean_train_score"]-1
    res["val_gini"] = 2*res["mean_test_score"]-1
    return res[["params","dev_gini","val_gini"]]


In [4]:
df = fill_null(df,-1)
df = cat_to_one_hot(df)

In [24]:
from hyperopt import hp
from hyperopt import fmin, tpe, hp, STATUS_OK, Trials
#XGBoost with GridSearch and CV
warnings.simplefilter('ignore', DeprecationWarning)
start_time = time.time()
x_sample=df.copy()
x_sample = x_sample.drop(['CONTRACT_SRC_CODE','SCORE_FINAL','BAD_12_FLAG90_1'], axis=1)
y = df["BAD_12_FLAG90_1"][df['BAD_12_FLAG90_1'].notnull()] 


In [48]:
#Random forest classifier
start_time = time.time()
from sklearn.ensemble import RandomForestClassifier
rf_model = RandomForestClassifier(bootstrap=True, class_weight=None, criterion='entropy',
            max_depth=2, max_features='auto', max_leaf_nodes=None,
            min_impurity_decrease=0.0, min_impurity_split=None,
            min_samples_leaf=1, min_samples_split=2,
            min_weight_fraction_leaf=0.0, n_estimators=10, n_jobs=-1,
            oob_score=False, random_state=0, verbose=0, warm_start=False)

kfold = KFold(n_splits=3, shuffle=True)
clf = GridSearchCV(rf_model,{'max_depth':[10],
                            'n_estimators':[100]},                             
                            scoring='roc_auc',
                            cv=kfold,
                            n_jobs=-1)
clf.fit(x_sample,y)

print(-start_time + time.time()," sec")
warnings.simplefilter('ignore', FutureWarning)
res = pd.DataFrame(clf.cv_results_)
res["dev_gini"] = 2*res["mean_train_score"]-1
res["val_gini"] = 2*res["mean_test_score"]-1
res[["params","dev_gini","val_gini"]]

building tree 1 of 100building tree 2 of 100
building tree 3 of 100
building tree 4 of 100building tree 5 of 100
building tree 6 of 100
building tree 7 of 100

building tree 8 of 100
building tree 9 of 100building tree 10 of 100
building tree 11 of 100


building tree 12 of 100
building tree 13 of 100
building tree 14 of 100
building tree 15 of 100
building tree 16 of 100
building tree 17 of 100
building tree 18 of 100
building tree 19 of 100
building tree 20 of 100
building tree 21 of 100
building tree 22 of 100
building tree 23 of 100
building tree 24 of 100
building tree 25 of 100
building tree 26 of 100
building tree 27 of 100
building tree 28 of 100
building tree 29 of 100
building tree 30 of 100
building tree 31 of 100
building tree 32 of 100


[Parallel(n_jobs=-1)]: Done  17 tasks      | elapsed:    6.5s


building tree 33 of 100
building tree 34 of 100
building tree 35 of 100
building tree 36 of 100
building tree 37 of 100
building tree 38 of 100
building tree 39 of 100
building tree 40 of 100
building tree 41 of 100
building tree 42 of 100
building tree 43 of 100
building tree 44 of 100
building tree 45 of 100
building tree 46 of 100
building tree 47 of 100
building tree 48 of 100
building tree 49 of 100
building tree 50 of 100
building tree 51 of 100
building tree 52 of 100
building tree 53 of 100
building tree 54 of 100
building tree 55 of 100
building tree 56 of 100
building tree 57 of 100
building tree 58 of 100
building tree 59 of 100
building tree 60 of 100
building tree 61 of 100
building tree 62 of 100
building tree 63 of 100
building tree 64 of 100
building tree 65 of 100
building tree 66 of 100
building tree 67 of 100
building tree 68 of 100
building tree 69 of 100
building tree 70 of 100
building tree 71 of 100
building tree 72 of 100
building tree 73 of 100
building tree 74

[Parallel(n_jobs=-1)]: Done 100 out of 100 | elapsed:   28.2s finished


,params,dev_gini,val_gini
0,"{'max_depth': 10, 'n_estimators': 100}",0.730412,0.599068


In [49]:
from hyperopt import hp
from hyperopt import fmin, tpe, hp, STATUS_OK, Trials
#XGBoost with GridSearch and CV
warnings.simplefilter('ignore', DeprecationWarning)
start_time = time.time()
x_sample=df.copy()
x_sample = x_sample.drop(['CONTRACT_SRC_CODE','SCORE_FINAL','BAD_12_FLAG90_1'], axis=1)
y = df["BAD_12_FLAG90_1"][df['BAD_12_FLAG90_1'].notnull()] 

X,Xcv,y,ycv=train_test_split(x_sample,y)
def objective(space):
    print(space)
    clf = xgb.XGBClassifier(n_estimators =500,colsample_bytree=space['colsample_bytree'],
                           learning_rate = 1,
                            max_depth = int(space['max_depth']),
                            min_child_weight = space['min_child_weight'],
                            subsample = space['subsample'],
                           gamma = space['gamma'],
                           reg_lambda = space['reg_lambda'],tree_method='gpu_hist')

    
    eval_set  = [( X, y), ( Xcv, ycv)]

    clf.fit(X, y,
            eval_set=eval_set, eval_metric="logloss",
            early_stopping_rounds=10,verbose=False)

    pred = clf.predict_proba(Xcv)[:,1]
    roc = roc_auc_score(ycv, pred)
    print("SCORE:", 2*roc-1)
    #change the metric if you like
    return {'loss':2*roc-1, 'status': STATUS_OK }


space ={'max_depth': hp.quniform("x_max_depth", 3, 3, 1),
        'min_child_weight': hp.quniform ('x_min_child', 1, 10, 1),
        'subsample': hp.uniform ('x_subsample', 0.7, 1),
        'gamma' : hp.uniform ('x_gamma', 0.1,0.5),
        'colsample_bytree' : hp.uniform ('x_colsample_bytree', 0.7,1),
        'reg_lambda' : hp.uniform ('x_reg_lambda', 0,1)
    }


trials = Trials()
best = fmin(fn=objective,
            space=space,
            algo=tpe.suggest,
            max_evals=100,
            trials=trials)

print(best)

{'colsample_bytree': 0.9342486935746763, 'gamma': 0.3195307017989014, 'max_depth': 3.0, 'min_child_weight': 5.0, 'reg_lambda': 0.06023262709228072, 'subsample': 0.7808634940026454}
SCORE: 0.6185580316116652
{'colsample_bytree': 0.9516428582892973, 'gamma': 0.47707393654995156, 'max_depth': 3.0, 'min_child_weight': 5.0, 'reg_lambda': 0.08105913447135049, 'subsample': 0.8413079457031777}
SCORE: 0.6225737597300078
{'colsample_bytree': 0.8797977591407671, 'gamma': 0.48222755509622606, 'max_depth': 3.0, 'min_child_weight': 6.0, 'reg_lambda': 0.6850628480934937, 'subsample': 0.976553861069996}
SCORE: 0.626979152548768
{'colsample_bytree': 0.833205895951346, 'gamma': 0.10212262425549841, 'max_depth': 3.0, 'min_child_weight': 9.0, 'reg_lambda': 0.2722145444242896, 'subsample': 0.9430090864499474}
SCORE: 0.6262957860419673
{'colsample_bytree': 0.8148218227487104, 'gamma': 0.3442407430506449, 'max_depth': 3.0, 'min_child_weight': 6.0, 'reg_lambda': 0.5778283009767423, 'subsample': 0.819191555775

KeyboardInterrupt: 

In [42]:
from hyperopt import hp
from hyperopt import fmin, tpe, hp, STATUS_OK, Trials
#XGBoost with GridSearch and CV
warnings.simplefilter('ignore', DeprecationWarning)
start_time = time.time()
x_sample=df.copy()
x_sample = x_sample.drop(['CONTRACT_SRC_CODE','SCORE_FINAL','BAD_12_FLAG90_1'], axis=1)
y = df["BAD_12_FLAG90_1"][df['BAD_12_FLAG90_1'].notnull()] 
X,Xcv,y,ycv=train_test_split(x_sample,y)
clf = xgb.XGBClassifier(n_estimators =200,colsample_bytree=1,
                           learning_rate = .3,
                            max_depth = 3,
                            min_child_weight = 1,
                            subsample = 1,
                           gamma = 0,
                           reg_lambda = 0,tree_method='gpu_hist')

    
eval_set  = [( X, y), ( Xcv, ycv)]

clf.fit(X, y,
        eval_set=eval_set, eval_metric="logloss",
        early_stopping_rounds=10,verbose=0)

pred = clf.predict_proba(Xcv)[:,1]
roc = roc_auc_score(ycv, pred)
print("SCORE:", 2*roc-1)
#change the metric if you like
print(pred)

SCORE: 0.6470085762519984
[0.00136349 0.01967971 0.0010819  ... 0.06580421 0.00885607 0.00187472]


In [37]:
y

243937    0
55485     0
60802     0
218307    0
119715    0
44661     0
267681    0
387987    0
597137    0
34057     0
476109    0
401749    0
123243    0
461379    0
406316    0
560069    0
288173    0
98382     0
112313    0
271309    0
235489    0
225579    0
341656    1
302606    0
86510     0
315664    0
114614    0
222043    0
248529    0
252279    0
         ..
489410    0
111219    0
306226    0
399998    0
340514    0
312639    0
432974    0
434940    0
272034    0
164946    0
253036    0
103456    0
479454    0
36399     0
19771     0
209255    0
90008     0
316115    0
227937    0
459477    0
76391     0
240898    1
39372     0
627829    0
214222    0
4851      0
97093     0
234522    0
563649    0
36328     0
Name: BAD_12_FLAG90_1, Length: 473280, dtype: uint8